В этом ноутбуке мы будем анализировать распределение токенов и их комбинаций в словах.

Используя токенизатор и N-граммный анализ, мы исследуем повторяющиеся паттерны, что поможет в задачах стемминга и лемматизации.

In [23]:
import os

from tokenizers import Tokenizer

tokenizer_uni = Tokenizer.from_file('../data/processed/tokenizer/words_unigram_5000.tokenizer.json')

In [24]:
with open('../data/processed/word_freqs/freq_1000000_oshhamaho.txt') as f:
    words = f.read().split('\n')

In [25]:
token_dist_dir = '../data/processed/token_distribution'
os.makedirs(token_dist_dir, exist_ok=True)

In [26]:
from collections import defaultdict
import nltk
from tqdm import tqdm


def create_token_ng_distribution(words, n=5):
    token_ngrams_freq_dist = nltk.FreqDist()
    ngrams_tokens = defaultdict(list)

    pgbar = tqdm(sorted(words), desc=f'create {n}-gram distribution')
    for word in pgbar:
        tokens = tokenizer_uni.encode(word).tokens
        token_ids = [tokenizer_uni.token_to_id(token) for token in tokens]

        ngrams = tuple(nltk.ngrams(tokens, n=n))
        token_ngrams_freq_dist.update(ngrams)
        for ng in ngrams:
            ngrams_tokens[ng].append((tokens, token_ids))

    return token_ngrams_freq_dist, ngrams_tokens

In [27]:
import csv
import pandas as pd


def is_valid_stem(stem, freq, min_freq=10, max_freq=100, min_len=3):
    # малая частота стема говорит о том что она возможно слишком специфичная и длинная
    if freq < min_freq:
        return False

    # большая частота говорит о том что она возможно слишком общая и короткая
    if freq > max_freq:
        return False

    # если длина стема слишком мала, кандидат тоже не подходит
    if len(stem) < min_len:
        return False

    # todo можно проверить на вхождение в список префиксов и суффиксов, так точность должна увеличиться
    return True


def save_by_stem(df_data, n, ng_name, freq):
    df = pd.DataFrame(df_data)
    # сортируя по количеству токенов в слове, наверху будут слова слова близкие к корню, а внизу сложносоставные слова
    df = df.sort_values('word_ng_len', ascending=True)

    f_name = f'({freq}){ng_name}'
    f_path = f'{token_dist_dir}/{n}/{f_name}.csv'
    df.to_csv(f_path, index=False, sep=',', quoting=csv.QUOTE_MINIMAL, header=True)


def choose_stem(token_ngrams_freq_dist, ngrams_tokens, n=5, is_save_by_stem=False):
    if is_save_by_stem:
        os.makedirs(f'{token_dist_dir}/{n}', exist_ok=True)

    data = []
    freq_dist_sorted = sorted(token_ngrams_freq_dist.items(), key=lambda x: x[1], reverse=True)
    pgbar = tqdm(freq_dist_sorted, desc=f'choose {n}-gram stem')
    for ng, freq in pgbar:
        # фильтруя n-граммы по частоте, получим часть слова которая встречается в составе других слов
        # то есть часть слова которую можно использовать для поиска похожих слов (стемминг)

        stem = ''.join(ng)
        if not is_valid_stem(stem, freq):
            continue

        df_data = []
        for _tokens, _token_ids in ngrams_tokens[ng]:
            word = ''.join(_tokens)
            df_data.append({
                'stem_ng_len': n,
                'stem': stem,
                'word_ng_len': len(_tokens),
                'word': word,
                'template': word.replace(stem, '?'*len(stem)),
                'tokens': '|'.join(_tokens),
            })

        if is_save_by_stem:
            ng_name = '_'.join(ng)
            save_by_stem(df_data, n, ng_name, freq)

        data.extend(df_data)

    return data

In [28]:
data = []
for n in range(1, 7):
    token_fd, ngrams_tokens = create_token_ng_distribution(words, n=n)
    data_i = choose_stem(token_fd, ngrams_tokens, n=n, is_save_by_stem=False)
    data.extend(data_i)

choose 6-gram stem: 100%|██████████| 191758/191758 [00:00<00:00, 2781917.67it/s]


In [29]:
stem_df = pd.DataFrame(data)
# Если стема полностью повторяет слово, она выбрана вполне удачно.
# В некоторых случаях возможно что это даже лемма, ну или поиск леммы сильно упрощается

# здесь не обязательно сравнивать строки полностью, достаточно количество токенов
stem_df['stem_is_full_word'] = stem_df['stem_ng_len'] == stem_df['word_ng_len']

sort_values = ['stem_ng_len', 'stem', 'word_ng_len', 'word']
stem_df = stem_df.sort_values(sort_values, ascending=True)
stem_df.to_csv(f'{token_dist_dir}/stem_candidates.csv.gz', index=False, sep=',', quoting=csv.QUOTE_MINIMAL, header=True, compression='gzip')

In [30]:
stem_df[stem_df['stem'] == 'джэгун'].head(n=30)

,stem_ng_len,stem,word_ng_len,word,template,tokens,stem_is_full_word
448267,2,джэгун,2,джэгун,??????,джэгу|н,True
448266,2,джэгун,3,Сыджэгун,Сы??????,Сы|джэгу|н,False
448268,2,джэгун,3,джэгунми,??????ми,джэгу|н|ми,False
448269,2,джэгун,3,джэгунри,??????ри,джэгу|н|ри,False
448270,2,джэгун,3,джэгунрэ,??????рэ,джэгу|н|рэ,False
448272,2,джэгун,3,дыджэгун,ды??????,ды|джэгу|н,False
448273,2,джэгун,3,зэрыджэгун,зэры??????,зэры|джэгу|н,False
448277,2,джэгун,3,уджэгун,у??????,у|джэгу|н,False
448280,2,джэгун,3,фызэрыджэгун,фызэры??????,фызэры|джэгу|н,False
448271,2,джэгун,4,джэгунрэт,??????рэт,джэгу|н|рэ|т,False


In [31]:
import re
from collections import Counter

replace_many_q = lambda x: re.sub(r'\?{1,}', '*', x)

templates_cnt = Counter([replace_many_q(tmpl) for tmpl in stem_df['template'].values])

In [32]:
templates_df = pd.DataFrame(templates_cnt.most_common(), columns=['template', 'freq'])
templates_df = templates_df[templates_df['freq'] > 10]

In [33]:
templates_df.to_csv(f'{token_dist_dir}/stem_templates.csv', index=False, sep=',', quoting=csv.QUOTE_MINIMAL, header=True)